# Epitome of Salt: Sunset Show Notes 
## Author: Kevin Brar
We are going to see how we did at rating the BLB cards before they came out on arena. 

## Guests
TODO

## DataSet
Link: TODO

Last updated: TODO

In [1]:
# Define raters and data file
raters = ['Kevin', 'Nate', 'Josh']
data = 'BLB_tier_list.csv'

In [2]:
import pandas as pd

# Function to read rater ratings
def read_ratings(rater):
    return pd.read_csv(f'data/{rater}.csv')

# Initialize eos DataFrame
eos = pd.DataFrame(columns=['Name', 'Color', 'Rarity'])

# Define the tier to numeric value mapping
tier_num = {'G': 0., 'D-': 1., 'D': 2., 'D+': 3., 
            'C-': 4., 'C': 5., 'C+': 6., 'B-': 7., 
            'B': 8., 'B+': 9., 'A-': 10., 'A': 11., 
            'A+': 12., 'F':0.}

# Read ratings for each rater and merge into eos DataFrame
is_first = True
for rater in raters:
    rater_data = read_ratings(rater)
    if is_first:
        eos['Name'] = rater_data['Name']
        eos['Color'] = rater_data['Color']
        eos['Rarity'] = rater_data['Rarity']
        is_first = False
    eos[rater] = rater_data['Rating']
    eos[f'{rater}_num'] = eos[rater].map(tier_num)

# Drop rows with any missing values
eos = eos.dropna().copy()

# Read the master tier list
master_list = pd.read_csv(f'data/{data}')

# Match names with master tier list and add tiers to eos
eos['Tier'] = eos['Name'].map(dict(zip(master_list['Name'], master_list['Tier'])))
eos['Tier_num'] = eos['Tier'].map(tier_num)

# Dropped entries due to insufficient data 
dropped_entries_initial = eos[eos.isna().any(axis=1)]
print(f"Initial entries to be dropped (due to missing data): {len(dropped_entries_initial)}")
print(dropped_entries_initial)

# Drop rows with missing Tier information
eos = eos.dropna().copy()

# Calculate rating errors for each rater
for rater in raters:
    eos[f'{rater}_error'] = abs(eos['Tier_num'] - eos[f'{rater}_num'])

# The final DataFrame
eos.to_csv('data/data.csv', index=False)
eos

Initial entries to be dropped (due to missing data): 1
          Name Color Rarity Kevin  Kevin_num Nate  Nate_num Josh  Josh_num  \
188  Wear Down     G      U    D-        1.0    D       2.0   D-       1.0   

    Tier  Tier_num  
188   SB       NaN  


,Name,Color,Rarity,Kevin,Kevin_num,Nate,Nate_num,Josh,Josh_num,Tier,Tier_num,Kevin_error,Nate_error,Josh_error
0,Brave-Kin Duo,W,C,C-,4.0,D+,3.0,D+,3.0,D+,3.0,1.0,0.0,0.0
1,Crumb and Get It,W,C,C-,4.0,C-,4.0,C,5.0,C+,6.0,2.0,2.0,1.0
2,Carrot Cake,W,C,C,5.0,C+,6.0,D,2.0,B+,9.0,4.0,3.0,7.0
3,Lifecreed Duo,W,C,C,5.0,C,5.0,C+,6.0,C,5.0,0.0,0.0,1.0
4,Nettle Guard,W,C,C+,6.0,C+,6.0,C-,4.0,C,5.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Starseer Mentor,M,U,B+,9.0,C-,4.0,B-,7.0,C+,6.0,3.0,2.0,1.0
237,Stormcatch Mentor,M,U,C,5.0,C,5.0,C,5.0,C,5.0,0.0,0.0,0.0
238,Vinereap Mentor,M,U,B-,7.0,B-,7.0,B-,7.0,A-,10.0,3.0,3.0,3.0
239,Seedglaive Mentor,M,U,B-,7.0,C+,6.0,B-,7.0,C,5.0,2.0,1.0,2.0


In [3]:
# See who's predictions did the best 
for rater in raters:
    error = eos[f'{rater}_error'].sum()
    print(f'{rater} error: {error}')

Kevin error: 394.0
Nate error: 419.0
Josh error: 418.0


In [4]:
# biggest misses
num = 10
for rater in raters:
    temp = eos.copy()
    temp = temp.sort_values(by=f'{rater}_error', ascending=False)
    print(temp[['Name', 'Tier', rater]][0:num])

                    Name Tier Kevin
118       Darkstar Augur   B+     G
79          Wishing Well   D-    B+
162   Festival of Embers    F    B-
77    Valley Floodcaller   C-    A-
37   Starfall Invocation    C     A
216     Patchwork Banner    B    D+
161        Sunspine Lynx   D+     B
155      Artist's Talent    F     C
144   Hoarder's Overflow   D-    C+
190     Clifftop Lookout   D+    B-
                     Name Tier Nate
157        Manifold Mouse    A   C-
200    Innkeeper's Talent   A+    C
205     Fecund Greenshell   A+   C+
33     Caretaker's Talent   B+   D+
111         Wick's Patrol   A-   C-
197      Mistbreath Elder   B-   D-
73   Stormchaser's Talent   B+   C-
16       Builder's Talent   B-    D
24       Harvestrite Host   B+   C-
37    Starfall Invocation    C   A-
                         Name Tier Josh
2                 Carrot Cake   B+    D
37        Starfall Invocation    C   A+
97         Bonecache Overseer   B-    G
111             Wick's Patrol   A-   C-
73      

In [5]:
# hottest takes
num = 10
for rater in raters:
    temp = eos.copy()
    other_raters = [x for x in raters if x != rater]
    temp[f'{rater}_hot'] = abs(((temp[[f'{x}_num' for x in other_raters]].sum(axis=1)) / (len(other_raters))) - temp[f'{rater}_num'])
    temp = temp.sort_values(by=f'{rater}_hot', ascending=False)
    top_num_cols = ['Name', 'Tier'] + [f'{x}' for x in raters]
    top_num = temp[top_num_cols].head(num)
    print(f'Top {num} hottest takes for {rater}:')
    print(top_num)

Top 10 hottest takes for Kevin:
                    Name Tier Kevin Nate Josh
79          Wishing Well   D-    B+    G    G
118       Darkstar Augur   B+     G   B-   B-
111        Wick's Patrol   A-    B+   C-   C-
114   Scavenger's Talent   C+     B    D    C
219         Fountainport    C    D-   C-   C+
162   Festival of Embers    F    B-   D+   D+
196  For the Common Good   D+    C+   D+   D-
240    Lilysplash Mentor   B-     B   C-    C
31     Essence Channeler    B    A-   C-   B+
207  Season of Gathering   A-     A   B-    B
Top 10 hottest takes for Nate:
                       Name Tier Kevin Nate Josh
197        Mistbreath Elder   B-     B   D-    B
157          Manifold Mouse    A    B+   C-    A
31        Essence Channeler    B    A-   C-   B+
79             Wishing Well   D-    B+    G    G
114      Scavenger's Talent   C+     B    D    C
119  Wick, the Whorled Mind   C+    C+   D+   B+
236         Starseer Mentor   C+    B+   C-   B-
17      Lupinflower Village   C+    B- 

In [6]:
# Calculate optimism or pessimism metric for each rater
for rater in raters:
    total_difference = (eos[f'{rater}_num'] - eos['Tier_num']).sum()
    if total_difference > 0:
        print(f'{rater} tends to be optimistic by {total_difference} points.')
    elif total_difference < 0:
        print(f'{rater} tends to be pessimistic by {-total_difference} points.')
    else:
        print(f'{rater} has no overall bias.')

Kevin tends to be optimistic by 34.0 points.
Nate tends to be pessimistic by 133.0 points.
Josh tends to be pessimistic by 100.0 points.
